In [25]:
pip install tensorflow

     -------------------------------------- 266.3/266.3 MB 2.2 MB/s eta 0:00:00
     ---------------------------------------- 1.5/1.5 MB 4.5 MB/s eta 0:00:00
     -------------------------------------- 439.2/439.2 KB 4.6 MB/s eta 0:00:00
     ---------------------------------------- 57.5/57.5 KB 3.0 MB/s eta 0:00:00
     -------------------------------------- 126.5/126.5 KB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 23.2/23.2 MB 3.1 MB/s eta 0:00:00
     ---------------------------------------- 6.0/6.0 MB 3.8 MB/s eta 0:00:00
     -------------------------------------- 895.7/895.7 KB 2.4 MB/s eta 0:00:00
     ---------------------------------------- 2.6/2.6 MB 2.5 MB/s eta 0:00:00
     ---------------------------------------- 93.3/93.3 KB 2.7 MB/s eta 0:00:00
     -------------------------------------- 781.3/781.3 KB 2.7 MB/s eta 0:00:00
     -------------------------------------- 232.7/232.7 KB 3.6 MB/s eta 0:00:00
     -------------------------------------- 15

You should consider upgrading via the 'C:\Users\rishu\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip' command.


In [43]:
import pandas as pd
df = pd.read_csv('train.csv')
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [44]:
df.isna().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [45]:
df = df.fillna('Not Available')
df['news'] = df['title'] +'\n '+ df['text']

In [46]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
sw = stopwords.words("english")
len(sw)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\rishu\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


179

In [52]:
import re
lines = []
k = 1000
for i in df.title:
    words = ""
    Q = i.lower()
    Q = re.sub("[^a-z ]","",Q)
    Q = Q.split(" ")
    #print(Q)
    for j in Q:      
        if(j not in sw):
            words = words + " " + j
    lines.append(words)

In [53]:
lines[0]

' house dem aide didnt even see comeys letter jason chaffetz tweeted'

In [62]:
from sklearn.feature_extraction.text import TfidfVectorizer
ti = TfidfVectorizer()
X = ti.fit_transform(pd.Series(lines)).toarray()
Y = df.label

MemoryError: Unable to allocate 3.54 GiB for an array with shape (20800, 22870) and data type float64

In [60]:
X.shape

(20800, 22870)

In [61]:
from sklearn.model_selection import train_test_split
xtrain, xtest, ytrain, ytest = train_test_split(X,Y,test_size=0.2,random_state=21,stratify=Y)

MemoryError: Unable to allocate 2.84 GiB for an array with shape (16640, 22870) and data type float64

In [40]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.callbacks import EarlyStopping
es = EarlyStopping(patience=10)
nn = Sequential()
nn.add(Dense(512,input_dim=22870,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(512,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(512,activation='relu'))
nn.add(Dropout(0.3))
nn.add(Dense(1,activation='sigmoid'))
nn.compile(loss='binary_crossentropy',metrics='accuracy',optimizer='adam')
model = nn.fit(xtrain,ytrain, validation_data=(xtest,ytest),callbacks=es,epochs=100)

MemoryError: Unable to allocate 1.42 GiB for an array with shape (16640, 22870) and data type float32

In [41]:
import matplotlib.pyplot as plt
plt.plot(model.history["accuracy"])
plt.plot(model.history["val_accuracy"])

NameError: name 'model' is not defined

In [ ]:
plt.plot(model.history["loss"])
plt.plot(model.history["val_loss"])

In [ ]:
tr_pred = nn.predict(xtrain)
ts_pred = nn.predict(xtest)

In [ ]:
tr_pred1 = []
for i in tr_pred:
    if i > 0.5:
        tr_pred1.append(1)
    else:
        tr_pred1.append(0)

In [ ]:
ts_pred1 = []
for i in ts_pred:
    if i > 0.5:
        ts_pred1.append(1)
    else:
        ts_pred1.append(0)

In [ ]:
from sklearn.metrics import accuracy_score
tr_acc = accuracy_score(ytrain,tr_pred1)
ts_acc = accuracy_score(ytest,ts_pred1)
print(tr_acc,ts_acc)

In [ ]:
from sklearn.metrics import confusion_matrix
cf = confusion_matrix(ytest,ts_pred1)
import seaborn as sb
sb.heatmap(cf,annot=True)

In [ ]:
from sklearn.metrics import classification_report
cr = classification_report(ytest,ts_pred1)
print(cr)

In [ ]:
df_test = pd.read_csv('test.csv')

In [ ]:
df_test.isna().sum()
df_test = df_test.fillna('Not Available')

In [ ]:
import re
lines_test = []
for i in df_test.title:
    words = ""
    Q = i.lower()
    Q = re.sub("[^a-z ]","",Q)
    Q = Q.split(" ")
    #print(Q)
    for j in Q:      
        if(j not in sw):
            words = words + " " + j
    lines_test.append(words)

In [ ]:
X_test = ti.transform(pd.Series(lines_test)).toarray()

In [ ]:
X_test

In [ ]:
test_pred = nn.predict(X_test)

In [ ]:
test_pred1 = []
for i in test_pred:
    if i>0.5:
        test_pred1.append(1)
    else:
        test_pred1.append(0)

In [ ]:
idx = list(df_test['id'])
df_sub = pd.DataFrame({'id':idx, 'label':test_pred1})

In [ ]:
df_sub.head()

In [ ]:
df_sub.groupby(by='label').count()

In [ ]:
df_sub.groupby(by='label').count().plot(kind='bar')

In [ ]:
df_sub.to_csv('submission.csv',index=False)